<img width="200" style="float:left" 
     src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Apache_Spark_logo.svg" />

# Sections
* [Description](#0)
* [1. Setup](#1)
  * [1.1 Start Hadoop](#1.1)  
  * [1.2 Search for Spark Installation](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Lab](#2)
  * [2.1 Check Twitter Files](#2.1)
  * [2.2 Create the DataFrame](#2.3)
  * [2.3 Perform Analytics](#2.3)
* [3. TearDown](#3)
  * [3.1 Stop Hadoop](#3.1)

<a id='0'></a>
## Description
<p>
<div>The goals for this lab are:</div>
<ul>    
    <li>Get familiar with Spark DataFrames API</li>
    <li>Apply some transformations using Spark DataFrames API</li>
</ul>    
</p>

<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS let's start the service

<a id='1.1'></a>
### 1.1 Start Hadoop

Start Hadoop

Open a terminal and execute
```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 1.2 Search for Spark Installation 
This step is required just because we are working in the course environment.

In [1]:
import findspark
findspark.init()

I'm changing pandas max column width property to improve data displaying

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

<a id='1.3'></a>
### 1.3 Create SparkSession

By setting this environment variable we can include extra libraries in our Spark cluster

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /opt/hive3/lib/hive-hcatalog-core-3.1.2.jar pyspark-shell'

The first thing always is to create the SparkSession

In [4]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Twitter - Analytics - DataFrames")
    .config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")
    .config("spark.sql.legacy.timeParserPolicy","LEGACY")
    .enableHiveSupport()
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<a id='2'></a>
## 2. Lab

<a id='2.1'></a>
### 2.1 Check Twitter Files

In order to complete this lab you need to previosly complete **'Twitter - RAW to STD - DataFrames'**.

Check you have the data ready in HDFS

http://localhost:50070/explorer.html#/datalake/std/twitter/bitcoin/

<a id='2.2'></a>
### 2.2 Create the DataFrame

The first step after creating the SparkSession is to create one or more DataFrames<br/>
The data in the std layer is often stored in advanced storage formats like **parquet** or **delta**.<br/>
These formats have the schema of the data embedded inside the file

In [5]:
tweets = (spark.read
               .parquet("hdfs://localhost:9000/datalake/std/twitter/ps5/"))

<a id='2.3'></a>
### 2.3 Perform Analytics

**Total number of tweets**<br/>


``` sql
select count(*)
from tweets
``` 

In [6]:
tweets.count()

2489

**Total number of distinct users**<br/>
``` sql
select count(distinct user.id)
from tweets
``` 

In [7]:
tweets.select("user.id").distinct().count()

2065

**Total number of users with geolocation enabled**<br/>
``` sql
select count(distinct user.id)
from tweets
where user.geo_enabled = true
``` 

In [18]:
tweets.where("user.geo_enabled=true").select("user.id").distinct()

DataFrame[id: bigint]

In [25]:
#Top geo locations

df = (tweets
      .where("user.geo_enabled=true")
      .select("place.country")
      .distinct())
df.toPandas()



,country
0,None
1,Ghana
2,United States
3,United Kingdom


In [27]:
#Number of tweets per geography 

df = (tweets
          .groupBy("place.country")
          .agg(max("user.statuses_count").alias("tweets_posted"))
          .orderBy(desc("tweets_posted"))
          .limit(10))
df.toPandas()      

,country,tweets_posted
0,None,1578565
1,United Kingdom,31948
2,United States,18753
3,Ghana,12928


In [30]:
tweets.toPandas().head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,retweet_count,favorite_count,entities,favorited,retweeted,possibly_sensitive,filter_level,lang,year,dt
0,2021-12-06 18:00:00,1467901649532379138,1467901649532379138,"@CrispsCold Ok but like they still do better than the competition, like you literally cannot play most ps1-3 games… https://t.co/lpkvnNXk0R","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",True,1.467900e+18,1467900027389591553,1.382412e+18,1382411562112192517,...,0,0,"([(CrispsCold,)], [], None, [(https://twitter.com/i/web/status/1467901649532379138,)], [])",False,False,None,low,en,2021,2021-12-06
1,2021-12-06 18:00:00,1467901649922396166,1467901649922396166,The launch of Godfall: Challenger edition has started rolling out globally! PlayStation users can can begin downloa… https://t.co/8XtDEb0nQE,"<a href=""https://studio.twitter.com"" rel=""nofollow"">Twitter Media Studio</a>",True,NaN,None,NaN,None,...,0,0,"([], [], None, [(https://twitter.com/i/web/status/1467901649922396166,)], [])",False,False,False,low,en,2021,2021-12-06
2,2021-12-06 18:00:00,1467901649758957568,1467901649758957568,@insomniacgames still hooking it up! https://t.co/CsmcAzjhHe,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,2.284110e+07,22841103,...,0,0,"([(insomniacgames,)], [], [(https://twitter.com/ecjaxson/status/1467901649758957568/photo/1,)], [], [])",False,False,False,low,en,2021,2021-12-06
3,2021-12-06 18:00:00,1467901650610188290,1467901650610188290,RT @MCU_Direct: Here are official screenshots of #SpiderManNoWayHome's Integrated suit and the Black &amp; Gold suit in #SpiderManPS5: https://…,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,None,NaN,None,...,0,0,"([(MCU_Direct,)], [(SpiderManNoWayHome,), (SpiderManPS5,)], None, [], [])",False,False,None,low,en,2021,2021-12-06
4,2021-12-06 18:00:00,1467901651000377345,1467901651000377345,He has a long history of burglaries. \n\nhttps://t.co/SPAJH5Bxxn,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",False,NaN,None,NaN,None,...,0,0,"([], [], None, [(https://www.wakefieldexpress.co.uk/news/crime/prolific-burglar-jailed-again-after-being-caught-on-cctv-stealing-playstation-3483255,)], [])",False,False,False,low,en,2021,2021-12-06
5,2021-12-06 18:00:00,1467901651776421895,1467901651776421895,RT @SupplyNinja: PS5 Digital will be dropping today for Total Tech Members!\n\nPS5 Digital \nhttps://t.co/ZSApbuzKrK \n\nPS5 Disc Link (probable…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,NaN,None,...,0,0,"([(SupplyNinja,)], [], None, [(http://bestbuy.7tiv.net/MXnPQq,)], [])",False,False,False,low,en,2021,2021-12-06
6,2021-12-06 18:00:00,1467901652447338501,1467901652447338501,🚨 PS5 Raffle #Announcement\n\nWe have partnered @MindsThirsty to bring you a PS5 raffle. Tickets are $5 ea there is n… https://t.co/N9dEl7iHMc,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",True,NaN,None,NaN,None,...,0,0,"([(MindsThirsty,)], [(Announcement,)], None, [(https://twitter.com/i/web/status/1467901652447338501,)], [])",False,False,False,low,en,2021,2021-12-06
7,2021-12-06 18:00:01,1467901653240197121,1467901653240197121,RT @YellingMadman: You guys have been amazing to me not only this year but for the last decade or more. I can't ever thank you guys enough!…,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,NaN,None,...,0,0,"([(YellingMadman,)], [], None, [], [])",False,False,None,low,en,2021,2021-12-06
8,2021-12-06 18:00:01,1467901654100062209,1467901654100062209,"RT @Sageville: Been playing #ChorusGame on #PS5, it's really good. \n\nIf you are a space shooter fan, this is one that makes flight feel vi

**Total number of tweets per language**<br/>
``` sql
select lang,count(*) as total
from tweets
group by lang
``` 

In [9]:
from pyspark.sql.functions import *

df = (tweets
      .groupBy("lang")
      .agg(count("*").alias("total")))
      
df.toPandas()

,lang,total
0,en,2489


**Top 10 users with more tweets posted**<br/>
``` sql
select user.screen_name, max(user.statuses_count) tweets_posted 
from tweets
group by user.screen_name
order by tweets_posted desc
limit 10
```


In [10]:
df = (tweets
          .groupBy("user.screen_name")
          .agg(max("user.statuses_count").alias("tweets_posted"))
          .orderBy(desc("tweets_posted"))
          .limit(10))
df.toPandas()

,screen_name,tweets_posted
0,falconhamada_90,1578565
1,littlebytesnews,1258480
2,BlazedRTs,969584
3,Sexual_hub2,965927
4,stream_caster,844705
5,TwitchSIE,794714
6,JasonLopezII,690867
7,DataAugmented,666820
8,ArgosHelpers,604629
9,RoseParamore7,593696


**Top 10 users with more followers**<br/>
``` sql
select user.screen_name, max(user.followers_count) follower_count 
from tweets
group by user.screen_name
order by followers_count desc
limit 10
```


In [11]:
df = (tweets
          .groupBy("user.screen_name")
          .agg(max("user.followers_count").alias("followers_count"))
          .orderBy(desc("followers_count"))
          .limit(10))
df.toPandas()

,screen_name,followers_count
0,PlayStation,23178552
1,BlizzardCS,1016519
2,Ella_exclusive,673692
3,ASRBABES,504568
4,Sexual_hub2,489629
5,Tanikaso1,417394
6,LeedsNews,234403
7,_Illicit_Still,207688
8,AskPS_UK,187897
9,B_flix_,155247


**Top 10 users with more mentions**<br/>
``` sql
select lower(user_mention) as user_mention, count(*) as mentions
from tweets lateral view explode(entities.user_mentions.screen_name) u as user_mention
group by lower(user_mention)
order by mentions desc
limit 10
```

In [12]:
df = (tweets
          .select(explode("entities.user_mentions.screen_name").alias("user"))
          .groupBy(lower("user"))
          .agg(count("*").alias("mentions"))
          .orderBy(desc("mentions"))
          .limit(10))
df.toPandas()

,lower(user),mentions
0,playstation,315
1,insomniacgames,194
2,spiderman,178
3,marvel,108
4,guerrilla,97
5,mcu_direct,60
6,staplesuk,53
7,unrealengine,52
8,marvelgames,46
9,consoledetail,34


**Top 10 more popular hashtags**<br/>
``` sql
select lower(hashtag) as hashtag, count(*) as total
from tweets lateral view explode(entities.hashtags.text) h as hashtag
group by lower(hashtag)
order by total desc
limit 10
```

In [13]:
df = (tweets
      .select(explode("entities.hashtags.text").alias("hashtag"))
      .groupBy("hashtag")
      .agg(count("*").alias("total"))
      .orderBy(desc("total"))
      .limit(10))
      
df.toPandas()

# to normalize (upper & lower case version of the same hashtag)
#.groupBy(lower("hashtag").alias("hashtag"))

,hashtag,total
0,SpiderManNoWayHome,442
1,PS4live,93
2,PS5,85
3,SpiderManPS5,62
4,PS5Share,51
5,WIN,26
6,StaplesUKPrizeDraw,26
7,PS4,24
8,ps5,20
9,Sony,13


**Top 10 more popular cashtags**<br/>
``` sql
select lower(hashtag) as hashtag, count(*) as total
from tweets lateral view explode(entities.symbols.text) h as hashtag
group by lower(hashtag)
order by total desc
limit 10
```

In [14]:
df = (tweets
    .select(explode("entities.symbols.text").alias("cashtag"))
    .groupBy(upper("cashtag").alias("cashtag"))
    .agg(count("*").alias("total"))\
    .orderBy(desc("total"))
    .limit(10))
    
df.toPandas()

,cashtag,total
0,NSRGY,1
1,SONY,1


**Average number of words per tweet**<br/>
``` sql
select avg(size(split(text, ' '))) as avg_words
from tweets
```

In [15]:
tweets.select(avg(size(split("text", " "))).alias("avg_words")).toPandas()

,avg_words
0,17.279229


**Max and average number of hashtags**<br/>
``` sql
select max(size(entities.hashtags)) as max,
	   avg(size(entities.hashtags)) as average
from tweets
```

In [16]:
(tweets.select(
            max(size("entities.hashtags")).alias("max"),
            avg(size("entities.hashtags")).alias("average")
)).toPandas()

,max,average
0,15,0.590197


You have to install emojis library <br/>
Open a terminal and execute
```sh
pip3 install emojis
```

**Top 20 more popular emojis**<br/>

```sql
select emoji, count(*) as total
from tweets lateral view explode(get_emojis_udf(text)) e as emoji
group by emoji
order by total desc
limit 20
```

In [17]:
from pyspark.sql.functions import udf

import emojis

@udf("array<string>")
def get_emojis_udf(s):
    set = emojis.get(s)
    return [*set, ]

tweets.select(explode(get_emojis_udf("text")).alias("emoji"))\
      .groupBy("emoji").agg(count("*").alias("total")).orderBy(desc("total")).limit(20)\
      .toPandas()

ModuleNotFoundError: No module named 'emojis'

<a id='3'></a>
## 3. Tear Down

Once we complete the the lab we can stop all the services

<a id='3.1'></a>
### 3.1 Stop Hadoop

Stop Hadoop

Open a terminal and execute
```sh
hadoop-stop.sh
```